In [26]:
pip install langdetect

In [27]:
# Import dependencies
from sklearn.preprocessing import LabelEncoder
from langdetect import detect, detect_langs
from google.colab import files
import pandas as pd
from itertools import islice
import time

In [28]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [29]:
# Shareable link
link = 'https://drive.google.com/file/d/1e2HjzljCHSDLqJWOMZ_noMzklSAgYTws/view?usp=sharing'
id = '1e2HjzljCHSDLqJWOMZ_noMzklSAgYTws'

In [30]:
# Read in DataFrame
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('song_lyrics_popularity.csv')  
song_lyrics_df = pd.read_csv('song_lyrics_popularity.csv')
song_lyrics_df.head(3)

,song,song_id,artist,artist_id,category,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1


In [31]:
# Remove the songs without lyrics
song_lyrics_df = song_lyrics_df.dropna()

In [32]:
# Add encoded category column
le = LabelEncoder()
new_column = le.fit_transform(song_lyrics_df['category']) 
song_lyrics_df.insert(5, 'category_id', new_column)

# Remove non_alpha_words column and null values
#song_lyrics_df = song_lyrics_df.drop('non_alpha_words', axis=1)
#song_lyrics_df = song_lyrics_df.dropna()

# Save DataFrame to CSV
song_lyrics_df.to_csv('encoded_df.csv', index=False)
#files.download('encoded_df.csv')
song_lyrics_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1


In [33]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (109 kB/s)

In [34]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lyrics_NLP').getOrCreate()

In [35]:
# Read in DataFrame
music_df = spark.read.csv('encoded_df.csv', sep=",", header=True)
music_df.show()


+--------------------+--------------------+--------------+--------------------+--------+-----------+----------+--------------------+---------------------+-------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+-------------------+------------------+--------------------+-----------------------+--------------------+---------------+
|                song|             song_id|        artist|           artist_id|category|category_id|popularity|              genres|audio_ft_danceability|    audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|   audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|
+--------------------+--------------------+--------------+--------------------+--------+-----------+----------+--------------------+----

In [36]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [37]:
# Tokenize lyrics
tokenizer = Tokenizer(inputCol='lyrics', outputCol='words')
tokenized_df = tokenizer.transform(music_df)
tokenized_df.show()

+--------------------+--------------------+--------------+--------------------+--------+-----------+----------+--------------------+---------------------+-------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+-------------------+------------------+--------------------+-----------------------+--------------------+---------------+--------------------+
|                song|             song_id|        artist|           artist_id|category|category_id|popularity|              genres|audio_ft_danceability|    audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|   audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|               words|
+--------------------+--------------------+--------------+--------------------+--------+------

In [38]:
# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='filtered')
removed_df = remover.transform(tokenized_df)
removed_df.show()

+--------------------+--------------------+--------------+--------------------+--------+-----------+----------+--------------------+---------------------+-------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+-------------------+------------------+--------------------+-----------------------+--------------------+---------------+--------------------+--------------------+
|                song|             song_id|        artist|           artist_id|category|category_id|popularity|              genres|audio_ft_danceability|    audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|   audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|               words|            filtered|
+--------------------+--------------------+---------

In [39]:
# Convert DataFrame to Pandas
nlp_df = removed_df.toPandas()
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou..."


In [40]:
# Remove instrumental songs
for index, row in nlp_df.iterrows():
    if row['lyrics'].lower() == 'instrumental':
        nlp_df = nlp_df.drop(index)
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou..."


In [41]:
# Create a list of languages
t0 = time.time()
languages = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    try:
        langs = detect_langs(' '.join(map(str, filtered_words)))
        languages.append((len(langs), str(langs[0])[0:2]))
    except:
        languages.append(None)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(languages))
len(set(languages))

Run time: 76.56315493583679 seconds
8444


69

In [42]:
# Find songs indeces with one language that is not English
no_lang = []
one_lang_not_english = []
mult_langs_not_english = []
for id, item in enumerate(languages):
  if item == None:
    no_lang.append(id)
  if item != None:
    if (item[0] == 1) & (item[1] != 'en'):
      one_lang_not_english.append(id)
    if (item[0] > 1) & (item[1] != 'en'):
      mult_langs_not_english.append(id)
print(len(no_lang))
print(len(one_lang_not_english))
len(mult_langs_not_english)

9
120


131

In [43]:
# Print songs with no language
for index in no_lang:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])

                   song artist lyrics
428  How Low Can You Go     LP    4.0
                song   artist lyrics
2245  Salomon's Gate  Beherit      ?
                 song    artist    lyrics
2345  Dialectic Chaos  Megadeth          
             song artist lyrics
3469  Lost on You     LP    4.0
                     song         artist  lyrics
4270  Baby Can I Hold You  Tracy Chapman  74.375
                    song         artist  lyrics
4296  Give Me One Reason  Tracy Chapman  100.22
                                song                  artist lyrics
5751  "Tosca / Act 2: ""Vissi d'arte  7llzoWbL1ToAHq9IL3F84D    3.0
                       song            artist    lyrics
5812  """We're Not Programs   We're People"""  310253.0
                                                 song  ... lyrics
5859  "Sonata No. 14 ""Moonlight"" in C-Sharp Minor""  ...    3.0

[1 rows x 3 columns]


In [44]:
# Print non English songs with multiple languages
for index in mult_langs_not_english:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])

                  song  ...                                             lyrics
21  My Head & My Heart  ...   Baby now and then I think about me now and wh...

[1 rows x 3 columns]
     song    artist                                             lyrics
34  Fever  Dua Lipa   Before you came around I was doing just fine ...
        song artist                                             lyrics
61  Your Man   Joji   Oh oh oh oh oh Oh oh oh oh oh Oh oh oh   Have...
                         song  ...                                             lyrics
109  Something Just Like This  ...   Ive been reading books of old The legends and...

[1 rows x 3 columns]
      song           artist                                             lyrics
168  mmmmm  Daisy the Great  Gimme one more day To bask in your light Now t...
                 song  ...                                             lyrics
175  Take You Dancing  ...   Da da da da da da Da da da da da da Da da da ...

[1 rows x 3 columns]
      

In [45]:
# Print non English songs with one language
for index in one_lang_not_english:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])
songs_to_keep = [78, 453, 1084, 1089, 1634, 2151, 2405, 2430, 2529, 2838, 2993, 3633, 3737, 3805, 3846, 4215, 4389, 4664, 5070, 5148,
                 5209, 5265, 5360, 5661, 5732, 5786, 5828, 5958, 6119, 6269, 6464, 6472, 6495, 6502, 6595, 6627, 6696, 6729, 7277]

                                   song  ...                                             lyrics
78  Keep It Simple (feat. Wilder Woods)  ...   We lost our love Somewhere in the darkness ov...

[1 rows x 3 columns]
            song artist                                             lyrics
140  Mango Negro  Rubio     Sintiendo que me voy Mis pies flotan En el ...
          song  ...                                             lyrics
161  Melocotón  ...   Deja que se evapore el ego Y haz eco De lo qu...

[1 rows x 3 columns]
          song     artist                                             lyrics
275  Baka Baka  emir taha   Bu duvarlara baka baka Duvarlara baka baka Bu...
                         song  ...                                             lyrics
707  Laisse moi (feat. Hamza)  ...     Juste laisse moi laisse moi laisse laisse l...

[1 rows x 3 columns]
        song artist                                             lyrics
716  Whoopty     CJ   Heavy on the SSO shit man Loyal

In [46]:
# Add a language column to the DataFrame and replace the filtered column
nlp_df['language'] = languages

In [47]:
# Remove songs with no language
for index, row in nlp_df.iterrows():
    if row['language'] == None:
        nlp_df = nlp_df.drop(index)
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough...","(1, en)"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,...","(1, en)"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou...","(1, en)"


In [50]:
# Find the number of songs that do not have English lyrics
len(nlp_df[nlp_df['language'].apply(lambda x: 'en' not in x)])

251

In [51]:
# Save DataFrame to CSV
nlp_df.to_csv('nlp_df.csv', index=False)
files.download('nlp_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create a list of co-occurring words
cooccurring_words = []
for index, row in nlp_df.iterrows():
    lyrics = row['filtered']
    while '' in lyrics:
      lyrics.remove('')
    while ' ' in lyrics:
      lyrics.remove(' ')
    for i in range(len(lyrics)-1):
      cooccurring_words.append(lyrics[i] + ' ' + lyrics[i+1])
print(len(cooccurring_words))
len(set(cooccurring_words))

1376873


485774

In [ ]:
# Create a list of uncommon words from the re-occurring words list
t0 = time.time()
unique_cooccurring_words = list(set(cooccurring_words))
count = 0
uncommon_words = []
for words in unique_cooccurring_words[0:10000]:
  freq = cooccurring_words.count(words)
  if freq < 100:
    uncommon_words.append(words)
  else: 
    count+=1
    print(words, freq)
t1 = time.time()
print(f'Run time to create list of uncommon words: {t1-t0} seconds')
print(f'{count} of the first 10,000 unique co-occurring words appeared at least 100 times.')

# Create a new list without the uncommon words
t0 = time.time()
new_list = [words for words in cooccuring_words if words not in uncommon_words]
t1 = time.time()
print(f'Run time to create new list: {t1-t0} seconds')
len(new_list)

dont leave 230
? well 126
ha ha 276
know feel 160
see ? 159
bye bye 151
lets go 320
im fine 114
left behind 108
day die 100
love youre 110
dont mean 192
Run time to create list of uncommon words: 230.91541814804077 seconds
12 of the first 10,000 unique co-occurring words appeared at least 100 times.
Run time to create new list: 326.42130851745605 seconds


1351422

In [ ]:
# Create DataFrame of co-occurring words
t0 = time.time()
unique_cooccurring_words = list(set(new_list))
print(len(unique_cooccurring_words))
cooccurring_words_df = pd.DataFrame(columns=unique_cooccurring_words)
cooccurring_words_df.loc[0] = 0
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
cooccurring_words_df

475786
Run time: 24.398764610290527 seconds


,dont leave,? well,ha ha,know feel,north 40,see ?,bye bye,lets go,im fine,tippin big,drip em,left behind,theyre standing,day die,snowstorm shot,song shade,love youre,dont mean,biology dont,along round,countin one,days years,aint haters,ashamed come,10 rescue,though done,ahhhhhhhhhhhhhh ohhhhhhh,much mind,second glass,room touched,felt outside,clouds countin,overpowered struggles,touches hand,kisses else,kill argue,mirrors beauty,peace highness,rare diamonds,rainbow cadillacs,...,baby dragging,movin back,easily get,lies use,word getting,165 stripped,yeah catch,sweating next,feeling underneath,positive negative,talking somebody,jaded stay,hurt put,well health,better face,rigid mob,get bounce,turns dark,overrated obsessed,lies long,demigods certain,copied never,true good,350 tank,life size,rhyme one,grown love,thinkin count,lovin different,cloud life,afford lingerie,gone missin,stage show,wave blood,bout though,music us,aching hearts,fuck everyday,wanting react,different view
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Fill DataFrame with co-occurring word frequencies
t0 = time.time()
for words in new_list:
  cooccurring_words_df.loc[0, words]+=1
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
cooccurring_words_df

,dont leave,? well,ha ha,know feel,north 40,see ?,bye bye,lets go,im fine,tippin big,drip em,left behind,theyre standing,day die,snowstorm shot,song shade,love youre,dont mean,biology dont,along round,countin one,days years,aint haters,ashamed come,10 rescue,though done,ahhhhhhhhhhhhhh ohhhhhhh,much mind,second glass,room touched,felt outside,clouds countin,overpowered struggles,touches hand,kisses else,kill argue,mirrors beauty,peace highness,rare diamonds,rainbow cadillacs,...,baby dragging,movin back,easily get,lies use,word getting,165 stripped,yeah catch,sweating next,feeling underneath,positive negative,talking somebody,jaded stay,hurt put,well health,better face,rigid mob,get bounce,turns dark,overrated obsessed,lies long,demigods certain,copied never,true good,350 tank,life size,rhyme one,grown love,thinkin count,lovin different,cloud life,afford lingerie,gone missin,stage show,wave blood,bout though,music us,aching hearts,fuck everyday,wanting react,different view
0,230,126,276,160,1,159,151,320,114,3,3,108,8,100,1,3,110,192,2,1,1,3,2,2,1,2,1,12,1,2,1,1,1,1,1,1,1,1,1,2,...,1,1,2,1,1,1,4,1,1,1,1,1,2,1,13,1,1,1,1,1,1,1,3,1,1,3,1,1,1,1,1,1,1,1,1,2,1,1,8,9


In [ ]:
# Check that the frequencies add up correctly
t0 = time.time()
total = sum(cooccurring_words_df.loc[0])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'Sum: {total}')
# should equal 1,351,422

Run time: 0.03715062141418457 seconds
Sum: 1351422


In [ ]:
# Remove columns of words that only appear once
cooccurring_words_df = cooccurring_words_df.drop(columns=cooccurring_words_df.columns[(cooccurring_words_df[0:1] == 1).any()])    
cooccurring_words_df

,dont leave,? well,ha ha,know feel,see ?,bye bye,lets go,im fine,tippin big,drip em,left behind,theyre standing,day die,song shade,love youre,dont mean,biology dont,days years,aint haters,ashamed come,though done,much mind,room touched,rainbow cadillacs,safe secret,floor floor,yeah need,count dark,memory know,also present,forgotten ive,even die,lies dream,wandering gravity,along always,two throw,stick code,time sensuality,piano mothers,bells peace,...,hold believe,know exactly,ooh traveler,passion blowin,high heel,never taken,sad friends,often doubt,hey jude,motherfucker die,fake best,since somebody,page yeah,strung heavens,ive housing,shots ya,viva la,wanted end,sense fear,ago strange,rather stand,hearts aint,gettin longer,dreams gone,running running,kall martialisk,peaceful future,hour cried,caught love,fooled come,oh cryin,easily get,yeah catch,hurt put,better face,true good,rhyme one,music us,wanting react,different view
0,230,126,276,160,159,151,320,114,3,3,108,8,100,3,110,192,2,3,2,2,2,12,2,2,2,7,72,4,2,3,2,4,3,2,7,6,10,3,3,3,...,7,25,2,2,6,3,4,3,9,24,3,4,2,2,3,2,21,2,3,2,2,2,2,10,92,5,2,3,14,3,2,2,4,2,13,3,3,2,8,9


In [ ]:
# Create filtered DataFrame with co-occuring words that appear at least 250 times
filtered_cooccurring_words_df = cooccurring_words_df.drop(columns=cooccurring_words_df.columns[(cooccurring_words_df[0:1]<250).any()])    
filtered_cooccurring_words_df = filtered_cooccurring_words_df.sort_values(by=[0], axis=1, ascending=False)
filtered_cooccurring_words_df

,ha ha,lets go,know like,youve got,uh uh,oh oh,yeah yeah,cause dont,get back,du du,cant take,one time,yeah know,never gonna,ya ya,brand new,? yeah,youre gonna,hey !,thats way,know im,know dont,feels like,like im,every time,im going,love love,know youre,dont want,baby dont,oh baby,doo doo,time time,ah ah,baby baby,lets get,? cause,oh yeah,let go,dont wanna,...,come back,yeah !,run away,know love,one thing,cause im,ill never,like like,come come,wont let,yeah oh,night long,im im,make feel,feel like,youre one,im feeling,cant see,dont let,dont stop,please dont,dont think,dont get,feel good,last night,love ?,every day,na na,dont make,love baby,ill take,yeah im,oh im,think im,wanna see,? dont,dont worry,love like,dont care,ooh ooh
0,276,320,253,437,391,4770,3231,332,283,301,262,404,275,314,268,255,343,397,268,288,1006,457,382,596,615,437,994,520,982,440,498,983,439,1032,481,304,291,1075,1238,1519,...,435,317,271,267,276,873,430,372,376,299,446,293,321,370,927,370,251,507,464,588,385,409,283,250,257,347,435,2425,366,283,304,547,326,502,294,477,256,385,530,2572


In [ ]:
# Save DataFrame to CSV
filtered_cooccurring_words_df.to_csv('top_cooccurring_words.csv', index=False)
files.download('top_cooccurring_words.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create dictionary of most common co-occurring words
t0 = time.time()
top_word_counts = list(filtered_cooccurring_words_df.loc[0])
top_cooccurring_words = dict(zip(list(filtered_cooccurring_words_df.columns), top_word_counts))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
dict(islice(top_cooccurring_words.items(), 10))

Run time: 0.4865748882293701 seconds


{'dont know': 2384,
 'dont wanna': 1519,
 'im gonna': 1208,
 'la la': 2343,
 'let go': 1238,
 'na na': 2425,
 'oh oh': 4770,
 'oh yeah': 1075,
 'ooh ooh': 2572,
 'yeah yeah': 3231}

In [ ]:
# TF-IDF
hashing = HashingTF(inputCol='filtered', outputCol='hashedValues', numFeatures=pow(2,18))
hashed_df = hashing.transform(removed_df)
idf = IDF(inputCol='hashedValues', outputCol='features')
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)
#rescaledData.select('words', 'features').show()
rescaledData.show()